This notebook exemplifies a simple market access estimation based on global datasets:

- **Global Friction Surface (Malaria Atlas Project)**  
see: https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2019


- **World Pop 1 km Population Grid**

### Required libraries

In [1]:
import sys, os
import pandas as pd
import geopandas as gpd
import numpy as np
import skimage.graph as graph
import rasterio as rio
from shapely.geometry import box, Point, Polygon
import matplotlib.pyplot as plt
# from tqdm import tqdm

sys.path.append("/home/wb514197/Repos/GOST_Urban")
sys.path.append("/home/wb514197/Repos/INFRA_SAP")
sys.path.append('/home/wb514197/Repos/GOSTnets')

import src.UrbanRaster as urban
import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc
from infrasap import aggregator
import GOSTnets.calculate_od_raw as calcOD

# Plotting
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from rasterio.plot import plotting_extent
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# import contextily as ctx

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Set up a data directory

In [3]:
iso3 = "SRB"
data_dir = "/home/wb514197/data/serbia"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [4]:
output_path = os.path.join(data_dir, "output")
if not os.path.exists(output_path):
    os.mkdir(output_path)

### 1. Clip global datasets to country extent

In [4]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
inG = rio.open(global_friction_surface)
inP = rio.open(global_population)

# Read in country bounds
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inB = gpd.read_file(global_bounds)
inB = inB.loc[inB['ISO3'] == iso3]
inB = inB.to_crs(inG.crs.to_string())

In [44]:
# Clip the travel raster to ISO3
out_travel_surface = os.path.join(data_dir, "TRAVEL_SURFACE.tif")
# rMisc.clipRaster(inG, inB, out_travel_surface)

# Clip the population raster to ISO3
out_pop_surface = os.path.join(data_dir, "POP_2020_NEW.tif")
# rMisc.clipRaster(inP, inB, out_pop_surface)

In [45]:
travel_surf = rio.open(out_travel_surface)
pop_surf = rio.open(out_pop_surface)

In [46]:
# Make sure that both rasters have the exact same resolution, crs, and number of pixels
out_pop_surface_std = os.path.join(data_dir, "POP_2020_NEW_STD.tif")
# rMisc.standardizeInputRasters(pop_surf, travel_surf, os.path.join(data_dir, "POP_2020_NEW_STD.tif"), data_type="C")

### 2. Calculate urban extents

- Population Density > 300 people per sq. km
- Total population > 5,000 people

In [71]:
calc_urban.calculateUrban?

In [67]:
calc_urban = urban.urbanGriddedPop(out_pop_surface_std)
urban_extents = calc_urban.calculateUrban(densVal=300, totalPopThresh=5000, 
                          print_message=iso3, verbose=True)

11:06:38	SRB: Read in urban data
11:06:38	SRB: Creating Shape 0


In [100]:
urban_extents = calc_urban.calculateUrban(densVal=300, totalPopThresh=5000, 
                          print_message=iso3, verbose=True, smooth=True, queen=False,
                                          raster=os.path.join(output_path, "raster_bool.tif"))

11:19:40	SRB: Read in urban data
11:19:40	SRB: Creating Shape 0


In [9]:
urban_extents.geometry = urban_extents.representative_point()
urban_extents.set_index("ID", inplace=True)

In [23]:
urban_extents = gpd.read_file(os.path.join(output_path, "urban_extents.shp"))
urban_extents.geometry = urban_extents.representative_point()

In [39]:
urban_extents.to_csv(os.path.join(output_path, "urban_extents_centroids.csv"))

In [203]:
urban_extents = pd.read_csv(os.path.join(output_path, "urban_extents_centroids.csv"), index_col=0)

In [204]:
from shapely.wkt import loads

In [205]:
urban_extents.geometry = urban_extents.geometry.apply(lambda x: loads(x))

In [206]:
urban_extents_gpd = gpd.GeoDataFrame(urban_extents, geometry='geometry', crs='EPSG:4326')

In [207]:
urban_extents = urban_extents_gpd.copy()

In [208]:
urban_extents = urban_extents.loc[urban_extents.Pop>100000].copy()

### 3. Prepare origins (population grid)

Convert from raster format to csv/geopandas data frame

In [197]:
pop_surf = rio.open(out_pop_surface_std)
pop = pop_surf.read(1, masked=True)

In [198]:
indices = list(np.ndindex(pop.shape))

In [199]:
xys = [pop_surf.xy(ind[0], ind[1]) for ind in indices]

In [200]:
res_df = pd.DataFrame({
    'spatial_index': indices, 
    'xy': xys, 
    'pop': pop.flatten()
})

### 3. Calculate travel time

Original data expressed in units of minutes required to travel one metre.  
*Convert to minutes required to travel one pixel*


In [210]:
# create MCP object
inG_data = travel_surf.read(1) * 1000 #
# Correct no data values
inG_data[inG_data < 0] = 99999999
mcp = graph.MCP_Geometric(inG_data)

In [211]:
# for each destination get cost of travel for every origin
for idx, dest in urban_extents.iterrows():
    dest_gdf = gpd.GeoDataFrame([dest], geometry='geometry', crs='EPSG:4326')
    res = ma.calculate_travel_time(travel_surf, mcp, dest_gdf)[0]
    res_df.loc[:,idx] = res.flatten()

In [49]:
# remove values where pop is 0 or nan
res_df = res_df.loc[res_df['pop']!=0].copy()
res_df = res_df.loc[~(res_df['pop'].isna())].copy()

In [50]:
od_cities = np.array(res_df[urban_extents.index])

In [216]:
geoms = [Point(xy) for xy in res_df.xy]

In [217]:
access_cities_results = gpd.GeoDataFrame(res_df, geometry=geoms, crs=urban_extents.crs)

### 5. Save results in raster format

Convert travel time to hours and get minimum travel cost to nearest city

In [218]:
access_cities_results = access_cities_results.apply(lambda x: x/60 if x.name in urban_extents.index else x)

In [219]:
access_cities_results.loc[:, "tt_min"] = access_cities_results[urban_extents.index].min(axis=1)

In [221]:
aggregator.rasterize_gdf(access_cities_results, 'tt_min', out_travel_surface, os.path.join(output_path,f"cities_100k_min_tt_v2.tif"))